### 금융문자 분석
- 데이터 제공이 안된다.... 
- 베이스 라인 코드 따라쳐보고 의미 파악 하는 방향으로..

In [1]:
import pandas as pd
import numpy as np
import random
from pandas import DataFrame
from collections import Counter

from tqdm import tqdm # 시간 측정

from sklearn.feature_extraction.text import CountVectorizer # 벡터라이저 모델
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

/home/loveactualry/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
# colab 에서 사용 하는 경우 명령어.
# colab을 써볼까??? 
# 환경 설정 걱정 안해도 될 것 같은데...
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# 변수에 값들 세주기...
Counter(train['smishing'])
# 결과를 보면 스미싱과 아닌 것의 개수가 Counter({0: 277242, 1: 18703})
# 비율 차이가 좀 있다.. 
# 산탄데르 문제와 비슷..?

In [ ]:
# 램을 덜 차지하게 비율을 찾아보는 과정
# under-sampling 방법
19646/train.shape[0]

In [ ]:
random.seed(2019) # 시드번호
train_nsm_list=list(train[train['smishg']!=1].index
# 0인 인덱스만 뽑아서 리스트를 만든다.
train_nsmishing=random.sample(train_nsm_list, 11750)
# 랜덤으로 11750개 샘플 뽑아서 저장.

In [ ]:
random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [ ]:
train_xx=train.iloc[train_smishing+train_nsmishing,:]\
.reset_index(drop=True)

# reset index  보기

In [ ]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [ ]:
# 데이터 프레임으로 데이터 넣어주기.test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [ ]:
# Mecab은 윈도우에서 작동하지 않는다.
# tokenizer를 활용하는 다른 함수.
# https://blog.pingpong.us/tokenizer/
# 해당 사이트 참조 - Mecab이 성능이 더 좋다는 얘기가 있다.

import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

In [ ]:
# 텍스트 토큰화 -> 토큰화를 하는 이유는 뭘까?
tokenizer = Mecab() # setting tokenizer using Mecab()

train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

In [ ]:
### 필요없는 단어를 제거해봅시다.

stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords # global을 왜 쓰나..?
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [ ]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [ ]:
# 학습 데이터 벡터화...
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
vec_x_test= v.transform(X_test).toarray()

In [ ]:
# 나이브 베이즈 multinomial nb
# 코드는 간단하구만..

In [ ]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_train)
y_train_pred1_one= [ i[1]  for i in y_train_pred1]

y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]